In [72]:
import os 
import cv2 # Computer vision -- traitement d'image
import numpy as np # array --> matrix de plus de deux dimention
import tkinter.filedialog

In [73]:
coin_datadir_train = '../coins-dataset/classified/train'
coin_datadir_test = '../coins-dataset/classified/test'
note_datadir_train = '../banknote-dataset/classified/train'
note_datadir_test = '../banknote-dataset/classified/test'

s_categories = ['coin', 'note']
categories = ['1c', '2c', '5c', '10c', '20c', '50c', '1e', '2e', '5e', '10e', '20e', '50e']
coin_index = 8

COIN OR NOTE

In [95]:
def resize_img(img):
    dim = (150, 150)
    new_img = cv2.resize(img, dim)
    return new_img

In [96]:
training_data = []


#add coins
for category in categories[:coin_index]:
    path = os.path.join(coin_datadir_train, category)
    label = 0
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        training_data.append([img_array, label])

#add notes
for category in categories[coin_index:]:
    path = os.path.join(note_datadir_train, category)
    label = 1
    for img in os.listdir(path):
        img_array = resize_img(cv2.imread(os.path.join(path, img)))
        training_data.append([img_array, label])


In [97]:
testing_data = []


#add coins
for category in categories[:coin_index]:
    path = os.path.join(coin_datadir_test, category)
    label = 0
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        testing_data.append([img_array, label])

#add notes
for category in categories[coin_index:]:
    path = os.path.join(note_datadir_test, category)
    label = 1
    for img in os.listdir(path):
        img_array = resize_img(cv2.imread(os.path.join(path, img)))
        testing_data.append([img_array, label])


In [99]:
x_train = []
y_train = []

for features, label in training_data:
    x_train.append(features)
    y_train.append(label)
    
x_train = np.array(x_train)

In [100]:
x_test = []
y_test = []

for features, label in testing_data:
    x_test.append(features)
    y_test.append(label)
    
x_test = np.array(x_test)

In [101]:
from collections import Counter
def k_nearest_neighbors(predict, k):
    distances = []
    for image in training_data:
        distances.append([np.linalg.norm(image[0] - predict), image[1]]) # calcul de distance euclidienne
    distances.sort()
    votes = [i[1] for i in distances[:k]]
    votes = ''.join(str(e) for e in votes)
    votes = votes.replace(',', '')
    votes = votes.replace(' ', '')
    result = Counter(votes).most_common(1)[0][0]
    return result

In [102]:
k_nearest_neighbors(x_train[0], 3)

'0'

In [103]:
from time import time
start = time()
correct = 0
total = 0
skipped = 0
for i in range(len(x_test)+1):
    try:
        prediction = k_nearest_neighbors(x_test[i], 5)
        if int(prediction) == y_test[i]:
            correct += 1
        total += 1
    except Exception as e:
        print('An exception occured')
        skipped += 1
accuracy = correct/total
end = time()
print(end-start)

Predicting
Correct
Predicting
Correct
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Correct
Predicting
Predicting
Predicting
Correct
Predicting
Predicting
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Predicting
Correct
Predicting
Predicting
Predicting
Correct
Predicting
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Correct
Predicting
Predicting
Correct
Pre

In [104]:
print(accuracy)

0.931454196028187
